In [1]:
#!pip install -q ultralytics
#!pip install opencv-python
#!pip install numpy
#!pip install mediapipe
#!pip install --upgrade mediapipe

In [2]:
import wget
wget.download('https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task')

'hand_landmarker (2).task'

In [3]:
from ultralytics import YOLO
# Load a model
# yolo11n.pt, yolo11s.pt, yolo11m.pt, yolo11l.pt, yolo11x.pt 分別對應到 YOLO tiny, small, medium, large, extra-large model
# yolo11n.pt, yolo11n-seg, yolo11n-pose, yolo11n-cls 分別進行 image localization, image segmentation, image keypoint detection, image classification
# model.track 進行物體追蹤
#
model = YOLO("yolo11n-pose.pt")  # load an official model
# model = YOLO("path/to/best.pt")  # load a custom model

# Predict with the model
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
#results = model.track(0, show=True)

for r in results:
    r.save()
    print(r.keypoints)

100%|██████████| 134k/134k [00:00<00:00, 741kB/s]


image 1/1 c:\Users\User\yolo_yo\predict\bus.jpg: 640x480 4 persons, 172.9ms
Speed: 10.6ms preprocess, 172.9ms inference, 13.4ms postprocess per image at shape (1, 3, 640, 480)
ultralytics.engine.results.Keypoints object with attributes:

conf: tensor([[0.9910, 0.9289, 0.9869, 0.4267, 0.9313, 0.9907, 0.9976, 0.9248, 0.9884, 0.9015, 0.9744, 0.9969, 0.9984, 0.9949, 0.9975, 0.9785, 0.9856],
        [0.1586, 0.1561, 0.0468, 0.2351, 0.0505, 0.6711, 0.2402, 0.5964, 0.1104, 0.4541, 0.1319, 0.7288, 0.5135, 0.7590, 0.5564, 0.5935, 0.4370],
        [0.9894, 0.9335, 0.9794, 0.5549, 0.9086, 0.9952, 0.9976, 0.9465, 0.9778, 0.9134, 0.9481, 0.9983, 0.9987, 0.9954, 0.9964, 0.9774, 0.9804],
        [0.0987, 0.0392, 0.0631, 0.0392, 0.0677, 0.2103, 0.2339, 0.2615, 0.3053, 0.3423, 0.3554, 0.2780, 0.2918, 0.2393, 0.2481, 0.1393, 0.1388]])
data: tensor([[[1.4236e+02, 4.4186e+02, 9.9095e-01],
         [1.4799e+02, 4.3142e+02, 9.2890e-01],
         [1.3054e+02, 4.3337e+02, 9.8691e-01],
         [0.0000e+00, 0.

In [8]:
import cv2 as cv
import numpy as np
from ultralytics import YOLO
import logging
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

logging.getLogger("ultralytics").setLevel(logging.ERROR)

model = YOLO("yolo11n-pose.pt")

# MediaPipe hand detector
model_path = r"C:\Users\User\yolo_yo\predict\hand_landmarker.task"
base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
detector = vision.HandLandmarker.create_from_options(options)

video_path = "WIN_20250520_21_42_55_Pro.mp4"
cap = cv.VideoCapture(video_path)
if not cap.isOpened():
    print("無法開啟影片")
    exit()

prev_wrist_x = {}

# ---------------- 手勢判斷 ---------------- #
def is_raising_hand(wrist, shoulder):
    if np.all(wrist == 0) or np.all(shoulder == 0):
        return False
    
    y_diff = shoulder[1] - wrist[1]
    x_diff = abs(wrist[0] - shoulder[0])
    return y_diff > 20 and x_diff < 200

def is_squatting(hip, knee):
    if np.all(hip == 0) or np.all(knee == 0):
        return False
    return (knee[1] - hip[1]) < 40

def is_waving_hand(track_id, wrist):
    if np.all(wrist == 0):
        return False
    x_now = wrist[0]
    waving = False
    if track_id in prev_wrist_x:
        diff = abs(x_now - prev_wrist_x[track_id])
        if diff > 20:
            waving = True
    prev_wrist_x[track_id] = x_now
    return waving

def is_ok_sign(landmarks):
    thumb_tip = np.array([landmarks[4].x, landmarks[4].y])
    index_tip = np.array([landmarks[8].x, landmarks[8].y])
    distance = np.linalg.norm(thumb_tip - index_tip)
    middle_tip = landmarks[12].y
    ring_tip = landmarks[16].y
    pinky_tip = landmarks[20].y
    palm_base = landmarks[0].y
    return distance < 0.05 and middle_tip < palm_base and ring_tip < palm_base and pinky_tip < palm_base

def is_peace_sign(landmarks):
    def is_extended(pip, tip):
        return tip.y < pip.y
    return (
        is_extended(landmarks[6], landmarks[8]) and
        is_extended(landmarks[10], landmarks[12]) and
        landmarks[16].y > landmarks[14].y and
        landmarks[20].y > landmarks[18].y
    )

def is_open_palm(landmarks):
    return all(landmarks[tip].y < landmarks[tip - 2].y for tip in [8, 12, 16, 20])

def is_fist(landmarks):
    wrist = np.array([landmarks[0].x, landmarks[0].y])
    closed_fingers = [np.linalg.norm(wrist - np.array([landmarks[i].x, landmarks[i].y])) < 0.1 for i in [8, 12, 16, 20]]
    return all(closed_fingers)

def is_clapping(hands):
    if len(hands) != 2:
        return False
    p1 = np.array([hands[0][5].x, hands[0][5].y])
    p2 = np.array([hands[1][5].x, hands[1][5].y])
    return np.linalg.norm(p1 - p2) < 0.08

# ---------------- 繪圖與手勢顯示 ---------------- #
def draw_landmarks_on_image(image, detection_result, thickness=3):
    annotated_image = image.copy()
    hands = detection_result.hand_landmarks
    clap_shown = is_clapping(hands) if hands else False

    for landmarks in hands or []:
        gesture = ""
        if is_ok_sign(landmarks):
            gesture = "OK Sign"
        elif is_peace_sign(landmarks):
            gesture = "Peace Sign"
        elif is_open_palm(landmarks):
            gesture = "Open Palm"
        elif is_fist(landmarks):
            gesture = "Fist"
        elif clap_shown:
            gesture = "Clap"

        for landmark in landmarks:
            x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
            cv.circle(annotated_image, (x, y), 7, (0, 0, 255), -1)
        connections = mp.solutions.hands.HAND_CONNECTIONS
        for connection in connections:
            start_idx, end_idx = connection
            start = landmarks[start_idx]
            end = landmarks[end_idx]
            start_point = (int(start.x * image.shape[1]), int(start.y * image.shape[0]))
            end_point = (int(end.x * image.shape[1]), int(end.y * image.shape[0]))
            cv.line(annotated_image, start_point, end_point, (0, 255, 0), thickness)

        if gesture and not clap_shown:
            x, y = int(landmarks[0].x * image.shape[1]), int(landmarks[0].y * image.shape[0])
            cv.putText(annotated_image, gesture, (x, y - 20), cv.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)

    if clap_shown:
        cv.putText(annotated_image, "Clap", (50, 80), cv.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

    return annotated_image

# ---------------- 主迴圈 ---------------- #
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("影片播放完畢")
        break

    results = model.track(frame, persist=True, tracker="botsort.yaml")

    if results:
        for r in results:
            annotated = r.plot()
            if hasattr(r, "keypoints") and r.keypoints is not None:
                kps = r.keypoints.xy.numpy()
                if len(kps) > 0:
                    for idx, kp in enumerate(kps):
                        track_id = int(r.id[idx]) if hasattr(r, "id") and r.id is not None else idx

                        lw = kp[9]; rw = kp[10]
                        lsh = kp[5]; rsh = kp[6]
                        lhip = kp[11]; rhip = kp[12]
                        lknee = kp[13]; rknee = kp[14]

                        left_up = is_raising_hand(lw, lsh)
                        right_up = is_raising_hand(rw, rsh)
                        squat_left = is_squatting(lhip, lknee)
                        squat_right = is_squatting(rhip, rknee)

                        frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
                        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)
                        detection_result = detector.detect(mp_image)
                        is_clap = is_clapping(detection_result.hand_landmarks if detection_result else [])

                        wave_left = is_waving_hand(f"{track_id}_L", lw) if not is_clap else False
                        wave_right = is_waving_hand(f"{track_id}_R", rw) if not is_clap else False

                        text = ""
                        if left_up and right_up:
                            text = "both hands "
                        elif left_up:
                            text = "left hand "
                        elif right_up:
                            text = "right hand "

                        if squat_left and squat_right:
                            text += "knees down "
                        if wave_left or wave_right:
                            text += "waving "

                        if text:
                            x, y = int(kp[0][0]), int(kp[0][1])
                            cv.putText(annotated, f"ID {track_id} {text}", (x, y - 10),
                                       cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

                        annotated_frame = draw_landmarks_on_image(annotated, detection_result)
                        cv.imshow("YOLO + MediaPipe Pose", cv.resize(annotated_frame, (900, 680)))

    if cv.waitKey(10) & 0xFF == ord(' '):
        break

cap.release()
cv.destroyAllWindows()


影片播放完畢
